In [1]:
# import relevant libraries
!pip install numerapi
!pip install ondemand
!pip install py_vollib
!pip install selenium
import os
import re
import sys
import glob
import json
import time
import math
import shutil
import config
import random
import pickle
import ondemand
import numerapi
import datetime
import py_vollib
import numpy as np
import scipy as sp
import pandas as pd
import requests as rq
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from os import system   
from math import floor
from scipy import stats
from copy import deepcopy
from datetime import datetime
from sklearn import linear_model
from sklearn import neural_network
from bs4 import BeautifulSoup as bs
from sklearn.decomposition import PCA
from statsmodels.formula.api import ols
from sklearn import metrics as sklMetrics
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

plt.style.use('seaborn')
%matplotlib inline
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 200)

ModuleNotFoundError: No module named 'config'

In [ ]:
# download the latest training dataset (~120 sec to run)
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
training_data.head()

In [ ]:
# download the latest tournament dataset (~120 sec to run)
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
tournament_data.head()

In [ ]:
# find only the feature columns
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]

In [ ]:
# select those columns out of the training dataset
training_features = training_data[feature_cols]

In [ ]:
# create a model and fit the training data (~30 sec to run)
model = neural_network.MLPRegressor(max_iter=500).fit(training_features, training_data.target)

In [ ]:
# select the feature columns from the tournament data
live_features = tournament_data[feature_cols]

In [ ]:
# predict the target on the live features
predictions = model.predict(live_features)

In [ ]:
# predictions must have an `id` column and a `prediction_kazutsugi` column
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions
predictions_df.head()

In [ ]:
# Get your API keys and model_id from https://numer.ai/submit
public_id = config.NuMeRaiPubKey
secret_key = config.NuMeRaiSecKey
model_id = config.NuMeRaiModId   
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [ ]:
# Upload your predictions
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)